## 1. Chuan bi van de (Prepare Problem)

### 1.1. Khai bao thu vien (Load Libraries)

In [1]:
import subprocess
import sys
import os
import json
import warnings
from IPython import display

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split


import openpyxl


%matplotlib inline
warnings.filterwarnings("ignore")
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

seed = 42
np.random.seed(seed)

print("Libraries loaded")

Libraries loaded


### 1.2. Tham so chuong trinh

In [ ]:
processed_data_dir = "../Data/processed_data"
scalers_dir = "../exps/scalers"

os.makedirs(processed_data_dir, exist_ok=True)
os.makedirs(scalers_dir, exist_ok=True)

print("=" * 70)
print("DATA PREPROCESSING SETUP")
print("=" * 70)
print(f"Processed data: {os.path.abspath(processed_data_dir)}")
print(f"Scalers:        {os.path.abspath(scalers_dir)}")
print("\nPurpose: Prepare data for K-means, Association Rules, and RF validation")


Processed data dir: c:\Users\ASUS\Documents\SGU-Data_Mining-Group\Projects\DTCK\Data\processed_data
Scalers dir: c:\Users\ASUS\Documents\SGU-Data_Mining-Group\Projects\DTCK\exps\scalers


### 1.3. Nap du lieu (Load Dataset)

In [3]:
df_org = pd.read_excel('../Data/Database paper.xlsx')

print("Raw data shape:", df_org.shape)
print("\nData types:")
print(df_org.dtypes)
print("\nFirst 5 rows:")
display.display(df_org.head())

Raw data shape: (2170, 22)

Data types:
Year                  int64
Gender                int64
Policy_Stu            int64
Minority_Stu          int64
Poor_Stu              int64
Father_Edu            int64
Mother_Edu            int64
Father_Occupation     int64
Mother_Occupation     int64
Time_Friends          int64
Time_SocicalMedia     int64
Time_Studying         int64
GPA                   int64
Adapt_Learning_Uni    int64
Study_Methods         int64
SupportOf_Uni         int64
SupportOf_Lec         int64
Facilitie_Uni         int64
Quality_Lecturer      int64
TrainingCurriculum    int64
Competitive_Class     int64
InfuenceF_Friends     int64
dtype: object

First 5 rows:


,Year,Gender,Policy_Stu,Minority_Stu,Poor_Stu,Father_Edu,Mother_Edu,Father_Occupation,Mother_Occupation,Time_Friends,...,GPA,Adapt_Learning_Uni,Study_Methods,SupportOf_Uni,SupportOf_Lec,Facilitie_Uni,Quality_Lecturer,TrainingCurriculum,Competitive_Class,InfuenceF_Friends
0,5,2,2,2,2,4,4,2,3,2,...,4,4,4,3,4,4,4,4,3,3
1,5,1,2,2,2,3,3,2,4,1,...,3,3,3,3,4,4,3,3,3,4
2,5,2,2,2,2,4,4,1,2,1,...,4,4,4,4,4,3,4,4,4,4
3,5,2,2,2,2,5,4,1,1,2,...,4,4,4,5,5,5,5,4,4,4
4,5,1,1,2,2,2,3,3,3,1,...,4,5,5,5,5,5,5,5,4,5


## 2. Lam sach du lieu (Data Cleaning)

In [4]:
# Check missing values
missing_count = df_org.isnull().sum()
if missing_count.sum() > 0:
    print(f"Found missing values, removing...")
    print(missing_count[missing_count > 0])
    df_org = df_org.dropna()
    print(f"New shape: {df_org.shape}")
else:
    print("No missing values")

# Check duplicates
dup_count = df_org.duplicated().sum()
if dup_count > 0:
    print(f"\nFound {dup_count} duplicate rows, removing...")
    df_org = df_org.drop_duplicates(ignore_index=True)
    print(f"New shape: {df_org.shape}")
else:
    print("\nNo duplicates")

# Basic statistics
print("\nBasic statistics:")
display.display(df_org.describe().round(2))

No missing values

Found 226 duplicate rows, removing...
New shape: (1944, 22)

Basic statistics:


,Year,Gender,Policy_Stu,Minority_Stu,Poor_Stu,Father_Edu,Mother_Edu,Father_Occupation,Mother_Occupation,Time_Friends,...,GPA,Adapt_Learning_Uni,Study_Methods,SupportOf_Uni,SupportOf_Lec,Facilitie_Uni,Quality_Lecturer,TrainingCurriculum,Competitive_Class,InfuenceF_Friends
count,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,...,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00,1944.00
mean,4.65,1.89,1.63,1.93,1.96,3.70,3.62,2.50,2.52,2.27,...,3.29,3.46,3.62,3.99,4.18,4.07,4.33,4.12,3.94,3.83
std,0.61,0.31,0.48,0.25,0.20,1.35,1.33,1.08,1.06,1.05,...,0.77,0.96,0.93,0.99,0.87,1.01,0.83,0.86,0.88,0.88
min,3.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,...,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00
25%,4.00,2.00,1.00,2.00,2.00,3.00,3.00,2.00,2.00,1.00,...,3.00,3.00,3.00,3.00,4.00,3.00,4.00,4.00,3.00,3.00
50%,5.00,2.00,2.00,2.00,2.00,4.00,4.00,3.00,3.00,2.00,...,3.00,3.00,4.00,4.00,4.00,4.00,5.00,4.00,4.00,4.00
75%,5.00,2.00,2.00,2.00,2.00,5.00,5.00,3.00,3.00,3.00,...,4.00,4.00,4.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00
max,5.00,2.00,2.00,2.00,2.00,6.00,6.00,5.00,5.00,5.00,...,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00,5.00


## 3. Xac dinh bien (Identify Features & Target)

In [5]:
target_col = 'GPA'
numeric_cols = [col for col in df_org.columns if col != target_col]

print(f"Target: {target_col}")
print(f"\nNumeric features ({len(numeric_cols)}):")
for i, col in enumerate(numeric_cols, 1):
    print(f"  {i}. {col}")

feature_info = {
    'numeric_cols': numeric_cols,
    'target_col': target_col,
    'n_features': len(numeric_cols)
}

with open(f'{processed_data_dir}/column_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)

print(f"\nFeature info saved")

Target: GPA

Numeric features (21):
  1. Year
  2. Gender
  3. Policy_Stu
  4. Minority_Stu
  5. Poor_Stu
  6. Father_Edu
  7. Mother_Edu
  8. Father_Occupation
  9. Mother_Occupation
  10. Time_Friends
  11. Time_SocicalMedia
  12. Time_Studying
  13. Adapt_Learning_Uni
  14. Study_Methods
  15. SupportOf_Uni
  16. SupportOf_Lec
  17. Facilitie_Uni
  18. Quality_Lecturer
  19. TrainingCurriculum
  20. Competitive_Class
  21. InfuenceF_Friends

Feature info saved


## 4. Chuan hoa du lieu cho EDA/PCA/KMeans (Data Standardization)

### 4.1. Chuan hoa tren FULL DATA (Fit on all data)

In [6]:
scaler_full = StandardScaler()
scaler_full.fit(df_org[numeric_cols])

df_scaled_full = df_org.copy()
df_scaled_full[numeric_cols] = scaler_full.transform(df_org[numeric_cols])

print(f"Scaler fitted on FULL data ({len(df_org)} rows)")
print(f"  Mean: {scaler_full.mean_[:5]}")
print(f"  Std: {scaler_full.scale_[:5]}")
print(f"\nScaled data shape: {df_scaled_full.shape}")
print(f"\nSample:")
display.display(df_scaled_full.head())

joblib.dump(scaler_full, f'{scalers_dir}/scaler_full_data.joblib')
df_scaled_full.to_excel(f'{processed_data_dir}/data_scaled_full.xlsx', index=False)
print(f"\nScaler & data saved")

Scaler fitted on FULL data (1944 rows)
  Mean: [4.65072016 1.88837449 1.63425926 1.93467078 1.95679012]
  Std: [0.60512174 0.31490516 0.48163726 0.24710587 0.20332925]

Scaled data shape: (1944, 22)

Sample:


,Year,Gender,Policy_Stu,Minority_Stu,Poor_Stu,Father_Edu,Mother_Edu,Father_Occupation,Mother_Occupation,Time_Friends,...,GPA,Adapt_Learning_Uni,Study_Methods,SupportOf_Uni,SupportOf_Lec,Facilitie_Uni,Quality_Lecturer,TrainingCurriculum,Competitive_Class,InfuenceF_Friends
0,0.577206,0.354473,0.759370,0.264377,0.212512,0.225106,0.283977,-0.459317,0.457636,-0.259736,...,4,0.561097,0.406536,-1.006348,-0.207256,-0.064787,-0.400171,-0.139131,-1.061911,-0.941052
1,0.577206,-2.821086,0.759370,0.264377,0.212512,-0.514094,-0.466093,-0.459317,1.402058,-1.210632,...,3,-0.480711,-0.670176,-1.006348,-0.207256,-0.064787,-1.611905,-1.304954,-1.061911,0.196638
2,0.577206,0.354473,0.759370,0.264377,0.212512,0.225106,0.283977,-1.382700,-0.486785,-1.210632,...,4,0.561097,0.406536,0.006251,-0.207256,-1.056480,-0.400171,-0.139131,0.071105,0.196638
3,0.577206,0.354473,0.759370,0.264377,0.212512,0.964307,0.283977,-1.382700,-1.431207,-0.259736,...,4,0.561097,0.406536,1.018850,0.943903,0.926907,0.811562,-0.139131,0.071105,0.196638
4,0.577206,-2.821086,-1.316882,0.264377,0.212512,-1.253295,-0.466093,0.464067,0.457636,-1.210632,...,4,1.602904,1.483248,1.018850,0.943903,0.926907,0.811562,1.026692,0.071105,1.334328



Scaler & data saved


### 4.2. Chuan bi du lieu Categorical cho Association Rules

In [7]:
def convert_to_categorical(df, numeric_cols):
    df_cat = df.copy()
    for col in numeric_cols:
        max_val = df_cat[col].max()
        if max_val == 5:
            df_cat[col] = pd.cut(df_cat[col], bins=[0, 2, 3.5, 5],
                                  labels=['Low', 'Medium', 'High'],
                                  include_lowest=True)
        elif max_val == 2:
            df_cat[col] = df_cat[col].map({1: 'No', 2: 'Yes'})
        else:
            df_cat[col] = df_cat[col].astype(str)
    return df_cat

df_categorical = convert_to_categorical(df_org, numeric_cols)

print(f"Categorical data shape: {df_categorical.shape}")
print(f"\nSample:")
display.display(df_categorical.head())

df_categorical.to_excel(f'{processed_data_dir}/data_categorical.xlsx', index=False)
print(f"\nCategorical data saved")

Categorical data shape: (1944, 22)

Sample:


,Year,Gender,Policy_Stu,Minority_Stu,Poor_Stu,Father_Edu,Mother_Edu,Father_Occupation,Mother_Occupation,Time_Friends,...,GPA,Adapt_Learning_Uni,Study_Methods,SupportOf_Uni,SupportOf_Lec,Facilitie_Uni,Quality_Lecturer,TrainingCurriculum,Competitive_Class,InfuenceF_Friends
0,High,Yes,Yes,Yes,Yes,4,4,Low,Medium,Low,...,4,High,High,Medium,High,High,High,High,Medium,Medium
1,High,No,Yes,Yes,Yes,3,3,Low,High,Low,...,3,Medium,Medium,Medium,High,High,Medium,Medium,Medium,High
2,High,Yes,Yes,Yes,Yes,4,4,Low,Low,Low,...,4,High,High,High,High,Medium,High,High,High,High
3,High,Yes,Yes,Yes,Yes,5,4,Low,Low,Low,...,4,High,High,High,High,High,High,High,High,High
4,High,No,No,Yes,Yes,2,3,Medium,Medium,Low,...,4,High,High,High,High,High,High,High,High,High



Categorical data saved


## 5. Luu du lieu da xu ly (Save Processed Data)

### Pipeline Summary:

1. **Load** dữ liệu thô (Database paper.xlsx)
2. **Clean** data (xóa missing, duplicates)
3. **Standardize** FULL DATA → `data_scaled_full.xlsx` (dùng cho K-means, PCA, RF validation)
4. **Categorize** features → `data_categorical.xlsx` (dùng cho Association Rules)
5. **Save** processed data

✓ Tất cả methods dùng cùng 1 dataset (2170 samples)  
✓ Không chia train/test vì mục đích validation, không phải prediction

In [ ]:
print("SAVING PROCESSED DATA")
print("="*70)

# 1. Scaled data (for K-means, PCA, RF validation)
df_scaled_full.to_excel(f'{processed_data_dir}/data_scaled_full.xlsx', index=False)
print(f"✓ Scaled data saved: data_scaled_full.xlsx")
print(f"  Shape: {df_scaled_full.shape}")
print(f"  Use for: K-means, PCA, Random Forest validation")

# 2. Categorical data (for Association Rules)
df_categorical.to_excel(f'{processed_data_dir}/data_categorical.xlsx', index=False)
print(f"\n✓ Categorical data saved: data_categorical.xlsx")
print(f"  Shape: {df_categorical.shape}")
print(f"  Use for: Association Rules mining")

# 3. Column info (for reference)
with open(f'{processed_data_dir}/column_info.json', 'w') as f:
    json.dump(feature_info, f, indent=2)
print(f"\n✓ Column info saved: column_info.json")
print(f"  Numeric features: {len(numeric_cols)}")
print(f"  Target: {target_col}")

# 4. Scaler (for reference)
joblib.dump(scaler_full, f'{scalers_dir}/scaler_full_data.joblib')
print(f"\n✓ Scaler saved: scaler_full_data.joblib")

print("\n" + "="*70)
print("✓ ALL DATA SAVED")
print("="*70)
print(f"\nProcessed data directory: {os.path.abspath(processed_data_dir)}")
print(f"\nFiles created:")
print(f"  1. data_scaled_full.xlsx    (StandardScaler, for K-means/PCA/RF)")
print(f"  2. data_categorical.xlsx    (Categorical, for Association Rules)")
print(f"  3. column_info.json         (Feature metadata)")
print(f"  4. scaler_full_data.joblib  (Fitted scaler)")


All data saved
